## **Authenticate Fabric & Assign to Variables**

In [ ]:
%run main.py

# Initiate FabricAuthOperations
fabric_auth = FabricAuthOperations()

# Get Authentication token and Save into 'token_store.json'
token = fabric_auth.get_bearer_token()

# Initiate OneLakeFileUtils
onelake_ops = OneLakeFileUtils()

# Initiate FabricAPIOperations
fabric_ops = FabricAPIOperations()

## **Import/Create Notebook**
**Reference:** [Create a notebook with a definition](https://learn.microsoft.com/en-us/fabric/data-engineering/notebook-public-api#create-a-notebook-with-a-definition)

In [ ]:
# Create Notebook in Workspace
fabric_ops.import_notebook_to_fabric(
    token=token,
    upload_from="github",  # ('local', 'lakehouse', or 'github')
    # source_path="Files/GitHub/notebooks/R in Spark/R_in_PySpark_Notebooks.ipynb", # lakehouse
    source_path="https://github.com/renan-peres/Polars-Cookbook/blob/main/Chapter07/ch07.ipynb", # github
    # source_path="test.ipynb", # local
    known_lakehouses=["3b9aef1c-59f2-47d9-a8b2-d8b234536427", "46389222-328e-4e65-aa06-02a380dd60d8"], # [LH_Test (Default), LH_bronze]
    # Optional parameters:
    # default_lakehouse_id="46389222-328e-4e65-aa06-02a380dd60d8", # LH_bronze
    # default_lakehouse_workspace_id="custom_workspace_id",
    # environment_id="custom_environment_id",
    # environment_workspace_id="custom_environmet_workspace_id"
)

## **Run Spark Notebook**
**Reference:** [Run a notebook on demand](https://learn.microsoft.com/en-us/fabric/data-engineering/notebook-public-api#run-a-notebook-on-demand)

In [ ]:
notebook_id = os.getenv("NOTEBOOK_ID")

fabric_ops.run_notebook_job(
    token=token,
    notebook_id=notebook_id,
    # Optional parameters:
    # workspace_id="your_workspace_id",
    # lakehouse_id="your_lakehouse_id",
    # lakehouse_name="your_lakehouse_name"
)

## **Run Data Pipeline**
**Reference:** [Microsoft Fabric data pipeline public REST API](https://learn.microsoft.com/en-us/fabric/data-factory/pipeline-rest-api)

In [ ]:
pipeline_id = os.getenv("PIPELINE_ID")

fabric_ops.trigger_pipeline_job(
    token=token,
    pipeline_id=pipeline_id,
    # Optional parameter:
    # workspace_id="3b9aef1c-59f2-47d9-a8b2-d8b234536427"
)

## **Run Delta Table Maintenance (REST API)**
**Reference:** [Background Jobs - Run On Demand Table Maintenance](https://learn.microsoft.com/en-us/rest/api/fabric/lakehouse/background-jobs/run-on-demand-table-maintenance?tabs=HTTP)

#### **Single Table**

In [ ]:
fabric_ops.trigger_table_maintenance_job(table_name="dim_coa_gold", token=token)

#### **All Tables**

In [ ]:
# Get the filtered subdirectory names for "Tables"
filtered_tables = onelake_ops.list_items(file_system_client=file_system_client, target_directory_path="Tables")

# Define batch size and delay between batches
batch_size = 5
batch_delay = 60  # in seconds

# Iterate over the filtered tables in batches
for i in range(0, len(filtered_tables), batch_size):
    batch_tables = filtered_tables[i:i + batch_size]
    for table_name in batch_tables:
        try:
            result = fabric_ops.trigger_table_maintenance_job(table_name=table_name, token=token)
            if result is not None:
                print(f"Table maintenance job triggered for table: {table_name}")
            else:
                print(f"Failed to trigger table maintenance job for table: {table_name}")
        except Exception as e:
            print(f"An error occurred for table {table_name}: {e}")
    
    # Delay between batches
    if i + batch_size < len(filtered_tables):
        print(f"Waiting for {batch_delay} seconds before triggering the next batch...")
        time.sleep(batch_delay)